### RAG Example with LangChain and Monitoring

**Two steps :** 
- create embedings and save to vector store
- create retriever and generation

**First Part**

In [ ]:
%load_ext autoreload
%autoreload 2


from src.config import Config
from src.document import DocumentLoader,DocumentProcessor
from src.vector_store import VectorStore
from src.llm import LLM

In [ ]:
config = Config()
config

In [ ]:
# Load documents
web_paths = [
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-01-10-inference-optimization/",
    "https://lilianweng.github.io/posts/2022-06-09-vlm/"
]
docs = DocumentLoader.load_web_documents(web_paths)


# # Process documents
processor = DocumentProcessor()

split_docs = processor.split_documents(docs)
# Create vector store
vector_store = VectorStore()


In [ ]:
vector_store.add_documents(split_docs)

**Second Part**

In [1]:
from src.rag import RAG
from src.config import Config
from src.vector_store import VectorStore
from src.llm import LLM

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
 # Initialize vector store
vector_store = VectorStore()
retriever = vector_store.get_retriever()

# Initialize LLM
llm = LLM().model

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [3]:
rag = RAG(retriever,llm)

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
INFO:src.db:Database initialized successfully
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [4]:
rag.run("What is ai agent")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections/0c748cc5-3d4e-4701-84c8-d693ac54f10b/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.db:Conversation 69c19c99-32b8-48fa-b428-b4552b8d91ac saved successfully
INFO:src.rag:RAG process completed for question: What is ai agent...


('An AI agent, particularly in the context of LLM-powered autonomous agents, functions as a problem solver using a large language model (LLM) as its core controller. It can break down complex tasks into manageable subgoals, reflect on past actions, and learn from mistakes to improve future performance. Additionally, it utilizes memory and external tools to enhance its capabilities and access information beyond its initial training.',
 {'answer': 'An AI agent, particularly in the context of LLM-powered autonomous agents, functions as a problem solver using a large language model (LLM) as its core controller. It can break down complex tasks into manageable subgoals, reflect on past actions, and learn from mistakes to improve future performance. Additionally, it utilizes memory and external tools to enhance its capabilities and access information beyond its initial training.',
  'model_used': 'gpt-4o-mini',
  'response_time': 2.177316,
  'relevance': 'RELEVANT',
  'prompt_tokens': 4,
  'c